In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [9]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'july'
new_month = 'august'

In [10]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/User/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2020.xlsx',\
                         sheet_name='assets',index_col='Ticker')

In [11]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [12]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.index = map(lambda x: x.lower(), df.index)

In [13]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

<ipython-input-13-5980899b9119>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_all = pd.Series()


In [14]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    if d_messari['data']['supply']['y_2050'] == None:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['circulating'])
    else:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Supply % Issued'] = df_all['Y2050 Supply'] / ((df_all['Y2050 Supply']))
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 180)
    except:
        pass
    return df_all

In [15]:
url_messari = "https://data.messari.io/api/v1/assets/elf/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)

In [16]:
a = df.index.tolist()

In [17]:
len(a)

142

In [35]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index[138:148]:
    try:
        print(t)
        metrics = get_messari_data(t)
        print(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
    except:
        pass

edo


<ipython-input-14-31327ca9407a>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_all = pd.Series()


edo
ela
ela
pai
pai
atom
atom


In [36]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [37]:
# t = 'ELF'
# metrics = get_messari_data(t)
# t = 'ELF_TEST'
# #print (metrics)
# metrics_log[t] = metrics
# metrics['Y2050 Supply']
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#    pass
# try:
#    df.loc[t,('Category2')] = metrics['Category2']
# except:
#    pass
# try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [38]:
metrics_log['BNT']

KeyError: 'BNT'

In [39]:
# t = 'BNT'
# metrics = get_messari_data(t)
# metrics_log[t] = metrics
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# #df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#     df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#     pass
# try:
#     df.loc[t,('Category2')] = metrics['Category2']
# except:
#     pass
# try:
#     df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#     pass

In [40]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [41]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [42]:
#read in other data sources
onchainfx_db = df

In [43]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

142
142
142
142


In [44]:
#fix any ticker mapping issues - rank date use these
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
onchainfx_db = onchainfx_db.rename({'IOT' : 'MIOTA'})
asset_db = asset_db.rename({'IOTA' : 'MIOTA'})

In [45]:
# #fix any ticker mapping issues - rebalance day use these
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
# asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [46]:
#onchainfx_db['Marketcap'] = np.where(onchainfx_db['Marketcap'].isnull() == True,onchainfx_db['Current Marketcap'],onchainfx_db['Marketcap'])

In [47]:
onchainfx_db.loc['BNT']

Coin                      Bancor
Class                          3
Sector                       NaN
Quality Exchange?              1
ERC20                          1
ICO ok?                        1
Eligible                       1
Y2050 Supply         6.91486e+07
Available Supply     6.91486e+07
Supply % Issued                1
Price                    1.60345
Marketcap            1.10876e+08
Current Marketcap    1.10876e+08
Est Volume           7.98758e+09
Name: BNT, dtype: object

In [50]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/User/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/User/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [51]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [52]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [53]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

In [54]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BCN,4.107269e+07,4.107269e+07,1.0,1.844675e+11,0.000223,1.844675e+11,7.730632e+06,1,0.188218,1,0.0,1.0,0.0,2.0,0
CENNZ,6.999492e+07,6.999492e+07,1.0,9.011038e+08,0.077677,9.011038e+08,1.191333e+07,1,0.170203,1,0.0,3.0,0.0,2.0,0
CTXC,3.071069e+07,3.071069e+07,1.0,2.539879e+08,0.120914,2.539879e+08,3.114319e+09,1,101.408295,1,0.0,2.0,0.0,2.0,0
DENT,2.439876e+07,2.439876e+07,1.0,1.000000e+11,0.000244,1.000000e+11,1.522581e+08,1,6.240403,1,0.0,3.0,0.0,2.0,0
ETN,5.071194e+07,5.071194e+07,1.0,1.017777e+10,0.004983,1.017777e+10,6.750870e+07,1,1.331219,1,0.0,3.0,0.0,2.0,0
GRIN,4.611081e+08,4.611081e+08,1.0,9.771011e+08,0.471914,9.771011e+08,1.299210e+09,1,2.817583,1,0.0,1.0,0.0,2.0,0
KCS,6.461605e+07,6.461605e+07,1.0,8.072839e+07,0.800413,8.072839e+07,1.435853e+09,1,22.221310,1,0.0,2.0,0.0,2.0,0
KIN,7.115478e+07,7.115478e+07,1.0,9.995502e+12,0.000007,9.995502e+12,1.808335e+07,1,0.254141,1,0.0,2.0,0.0,2.0,0
MKR,5.590322e+08,5.590322e+08,1.0,9.966470e+05,560.912989,9.966470e+05,3.484234e+09,1,6.232617,1,0.0,3.0,0.0,2.0,0
PLR,2.100994e+07,2.100994e+07,1.0,8.000000e+08,0.026262,8.000000e+08,9.643500e+06,1,0.458997,1,0.0,3.0,0.0,2.0,0


In [55]:
#eligible_db = eligible_db.append(ineligible_db.loc['XTZ'])
#eligible_db = eligible_db.drop('ATOM')

In [56]:
ineligible_db

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BCN,4.107269e+07,4.107269e+07,1.0,1.844675e+11,0.000223,1.844675e+11,7.730632e+06,1,0.188218,1,0.0,1.0,0.0,2.0,0
BLZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,2.0,0.0,1.0,0
CENNZ,6.999492e+07,6.999492e+07,1.0,9.011038e+08,0.077677,9.011038e+08,1.191333e+07,1,0.170203,1,0.0,3.0,0.0,2.0,0
CNX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,3.0,0.0,0.0,0
CTXC,3.071069e+07,3.071069e+07,1.0,2.539879e+08,0.120914,2.539879e+08,3.114319e+09,1,101.408295,1,0.0,2.0,0.0,2.0,0
DENT,2.439876e+07,2.439876e+07,1.0,1.000000e+11,0.000244,1.000000e+11,1.522581e+08,1,6.240403,1,0.0,3.0,0.0,2.0,0
DRGN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,3.0,0.0,0.0,0
EDGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,3.0,1.0,1.0,0
ETN,5.071194e+07,5.071194e+07,1.0,1.017777e+10,0.004983,1.017777e+10,6.750870e+07,1,1.331219,1,0.0,3.0,0.0,2.0,0
GBYTE,2.093438e+07,2.093438e+07,1.0,7.516807e+05,27.850100,7.516807e+05,1.516759e+06,1,0.072453,0,1.0,2.0,0.0,2.0,0


In [57]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

<ipython-input-52-d9c7362f0463>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)


In [58]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOT' : 'MIOTA'})
universe = universe.rename({'VEN' : 'VET'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,2.304165e+11,2.304165e+11,1.0,2.098634e+07,10979.359648,2.098634e+07,4.469887e+12,1,19.399161,1,1.0,1.0,0.0,3.0,1,1
ETH,4.303520e+10,4.303520e+10,1.0,1.351356e+08,318.459317,1.351356e+08,1.877088e+12,1,43.617491,1,1.0,2.0,0.0,3.0,1,2
XRP,2.434114e+10,2.434114e+10,1.0,1.000000e+11,0.243411,1.000000e+11,3.302981e+11,1,13.569542,1,1.0,2.0,0.0,3.0,1,3
ATOM,8.151951e+09,8.151951e+09,1.0,2.192096e+09,3.718792,2.192096e+09,1.703440e+10,1,2.089610,1,1.0,2.0,0.0,3.0,1,4
LINK,7.297230e+09,7.297230e+09,1.0,1.000000e+09,7.297230,1.000000e+09,9.983245e+10,1,13.680868,1,1.0,3.0,1.0,3.0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GAME,6.648562e+06,6.648562e+06,1.0,9.762285e+07,0.068105,9.762285e+07,7.814897e+06,1,1.175427,1,1.0,3.0,1.0,3.0,1,109
SNGLS,5.935486e+06,5.935486e+06,1.0,5.999999e+08,0.009892,5.999999e+08,3.209032e+07,1,5.406520,1,1.0,3.0,0.0,3.0,1,110
MLN,5.736144e+06,5.736144e+06,1.0,4.170070e+05,13.755511,4.170070e+05,1.301789e+08,1,22.694490,1,1.0,3.0,1.0,3.0,1,111
DCN,3.317510e+06,3.317510e+06,1.0,6.977108e+11,0.000005,6.977108e+11,1.467116e+07,1,4.422342,1,1.0,3.0,1.0,3.0,1,112


In [59]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained LTC
Retained XLM
10 full! No room for BSV
10 full! No room for EOS


In [60]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained RVN
Retained ZIL
Retained ZRX
Retained BAT
20 full! No room for DCR
20 full! Do not add  WAVES
20 full! Do not add  GNO
20 full! Do not add  LSK


In [61]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained IOST
Retained AE
Retained STEEM
Retained STORJ
Retained PIVX
Retained MANA
Retained GNT
Retained ELF
Retained ARDR
Retained WAN
40 full! No room for RCN
40 full! No room for GXC
40 full! No room for MAID
40 full! Do not add  EDO
40 full! Do not add  ANT
40 full! Do not add  STRAT
40 full! Do not add  WAX
40 full! Do not add  SYS
40 full! Do not add  FUN
40 full! Do not add  QASH


In [62]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [63]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [64]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

10 adds ['ADA' 'ATOM']
10 deletes ['BSV' 'EOS']
10 count members: 10

20 adds ['BSV' 'EOS']
20 deletes ['ADA' 'DCR']
20 count members: 20

40 adds ['BNT' 'DCR' 'GNO']
40 deletes ['GXC' 'MAID' 'RCN']
40 count members: 40

Total adds ['ATOM' 'BNT' 'GNO']
Total deletes ['GXC' 'MAID' 'RCN']
Total count members: 70

ETH adds ['BNT' 'GNO']
ETH deletes []
ETH count members: 22

Currency adds []
Currency deletes []
Currency count members: 19

Platform adds ['ATOM']
Platform deletes []
Platform count members: 27

Application adds ['BNT' 'GNO']
Application deletes ['GXC' 'MAID' 'RCN']
Application count members: 24

Sector count members: 70


In [65]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    if 'MIOTA' in indexes_new_tickers[k].index:
        indexes_new_tickers[k] = indexes_new_tickers[k].rename({'MIOTA' : 'IOT'})
    indexes_new_tickers[k].to_csv('C:/Users/User/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])